# Counting Distinct Elements in a Stream

Another common problem is counting the number of distinct elements that we have seen in the stream so far. Again the assumption here is that the universal set of all elements is too large to keep in memory, so we'll need to find another way to count how many distinct values we've seen. If we're okay with simply getting an estimate instead of the actual value, we can use the Flajolet-Martin (FM) algorithm.

## Flajolet-Martin Algorithm

In the FM algorithm we hash the elements of a strea into a bit string. A bit string is a sequence of zeros and ones, such as 1011000111010100100. A bit string of length $N$ can hold $2^N$ possible combinations. For the FM algorithm to work, we need $N$ to be large enough such that the bit string will have more possible combinations than there are elements in the universal set. This basically means that there should be no possible collisions when we has the elements into a bit string. 

The idea behind the FM algorithm is that the more distinct elements we see, the higher the likelihood that one of their hash values will be "unusual". The specific "unusualness" we will exploit here is that the bit string ends in many consecutive 0s.

For example, the bit string 1011000111010100100 ends with 2 consecutive zeros. We call this value of 2 the *tail length* of the bit string. Now let $R$ be the maximum tail length of that we have seen of any hashed bit string of the stream. The estimate of the number of distinct elements using FM is simply $2^R$.

In [19]:
#Library function for non-streams
def flajoletMartin(iterator):
    max_tail_length = 0
    
    for val in iterator:
        bit_string = bin(hash(hashlib.md5(val.encode('utf-8')).hexdigest()))
        
        i = len(bit_string) - 1
        tail_length = 0
        while i >= 0:
            if bit_string[i] == '0':
                tail_length += 1
            else:
                #neatly handles the '0b' prefix of the binary string too. 
                #Just break when we see "b"
                break
                
            i -= 1
            
        if tail_length > max_tail_length:
            max_tail_length = tail_length
            
    return (2**max_tail_length)

testList = []
n=0
while n < 100000:
    n += 1
    testList.append(np.random.choice(words))

print(flajoletMartin(iter(testList)))

65536


## Using Multiple Hash Functions

We can improve the estimate further by using multiple hash functions. With multiple hash functions we first split hash functions into groups, get the maximum tail length for each hash function, then get the average for each group. Lasly, we get the median over all the averages and that will be our estimate.

This way we can get estimates that aren't just powers of 2. If the correct count is between two large powers of 2, for example 7000, it will be impossible to get a good estimate using just one hash function.